In [ ]:
import json
# import glob
from tqdm import tqdm
import numpy as np
import unidecode
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet, stopwords, words
import re
import contractions
from bs4 import BeautifulSoup
import pandas as pd
from nltk.metrics.distance  import edit_distance
import emoji
import plotly.express as px

In [62]:
messages = open('./Data/result.json', 'r', encoding='utf8')
data = json.load(messages)

## Preprocessing

In [63]:
len(data['messages'][10000].keys())

7

In [123]:
def dataset_extraction(data):
    df = []
    crypto = ['shib', 'doge', 'shiba', 'dogecoin']
    print("Extracting Messages...")
    for i in tqdm(range(len(data['messages']))):
        frame = []
        st = data['messages'][i]
        
        if type(st['text']) == str:
            if any(curr in st['text'].lower() for curr in crypto):
                frame.append(st['date'].split("T")[0])
                frame.append(st['text'])

        if type(st['text']) == list: 
            b = str()
            
            for j in st['text']: 
                if type(j) == dict:
                    a = j['text']
                    b += a
                else: 
                    b += j
                    
            if any(curr in b.lower() for curr in crypto):
                frame.append(st['date'].split("T")[0])
                frame.append(b)
        
        if len(frame): 
            df.append(frame)
        

    return df

In [124]:
messages = dataset_extraction(data)

Extracting Messages...









  0%|                                                                        | 0/49436 [00:00<?, ?it/s]






  0%|                                                              | 17/49436 [00:00<06:05, 135.28it/s]






  0%|                                                              | 53/49436 [00:00<04:58, 165.60it/s]






  0%|▏                                                            | 117/49436 [00:00<03:52, 211.74it/s]






  1%|▍                                                            | 350/49436 [00:00<02:49, 290.38it/s]






  1%|▊                                                            | 635/49436 [00:00<02:02, 396.79it/s]






  2%|█                                                            | 838/49436 [00:00<01:33, 521.43it/s]






  2%|█▏                                                           | 996/49436 [00:00<01:14, 647.27it/s]






  3%|█▋                                                          | 1355/49436 [00:00<00:56, 854.43it/s]






  4

In [125]:
def convert_data_to_df(messages): 
    df = pd.DataFrame(messages, columns = ['Date', 'Messages'])
    return df

In [126]:
df = convert_data_to_df(messages)

In [127]:
pre = df.copy()

In [128]:
pre

,Date,Messages
0,2021-05-01,Doge is going craY
1,2021-05-01,Sell target of doge
2,2021-05-01,Doge
3,2021-05-01,Dogecoin!!! Que hago?
4,2021-05-01,"Anyway, is doge a good crypto for long term in..."
5,2021-05-01,Doge target 🎯
6,2021-05-01,Who else going to mars With doge😁
7,2021-05-01,Doge to the moon
8,2021-05-01,Doge is dompig 😞😞😞
9,2021-05-01,Doge 🐶 😂😅


In [141]:
def preprocessing(df):
    df = demojizing(df)
    df = convert_accented_chars(df)
    df = remove_case_sensitive(df)
    df = remove_htmls_and_urls(df)
    df = remove_extra_spaces_between_words(df)
    df = expand_contractions(df)
    df = remove_stop_words(df)
    df = lemmatization(df)
    df = removing_non_english(df)
    df = spelling_corrections(df)
    return df

def demojizing(dataframe): 
    tqdm.pandas()
    print("Decoding Emojis in Text...")
    dataframe["Messages"] = dataframe["Messages"].progress_apply(lambda txt: emoji.demojize(txt))
    return dataframe

def convert_accented_chars(dataframe): 
    tqdm.pandas()
    print("Converting Accented Characters...")
    dataframe["Messages"] = dataframe["Messages"].progress_apply(lambda txt: unidecode.unidecode(txt))
    return dataframe

def remove_case_sensitive(dataframe):
    tqdm.pandas()
    print("Removing Case Sensitive Characters...")
    dataframe['Messages'] = dataframe['Messages'].progress_apply(lambda txt: str(txt).lower())
    return dataframe

def remove_htmls_and_urls(dataframe):
    tqdm.pandas()
    print("Removing HTMLs and URLs...")
    dataframe['Messages'] = dataframe['Messages'].progress_apply(lambda txt: re.sub(r"http\S+", "", txt))
    dataframe['Messages'] = dataframe['Messages'].progress_apply(lambda txt: BeautifulSoup(txt, 'lxml').get_text())
    return dataframe
    
def remove_extra_spaces_between_words(dataframe):
    tqdm.pandas()
    print("Removing Extra Whitespaces...")
    dataframe['Messages'] = dataframe['Messages'].progress_apply(lambda txt: re.sub(" +"," ", txt))
    return dataframe

def expand_contractions(dataframe):
    tqdm.pandas()
    print("Expanding Contractions...")
    dataframe['Messages'] = dataframe['Messages'].progress_apply(lambda txt: contractions.fix(txt))
    return dataframe

def spelling_corrections(dataframe):
    
    def spell_check(message, correct_words):
        new_message = ""
        for word in message.split(" "):
            if word.isalpha() and (word not in correct_words) and (word.lower() not in ["doge", "dogecoin", "shibe", "shiba", "shib", "shiba inu"]):
                temp = [(edit_distance(word, w),w) for w in correct_words if w[0]==word[0]]
                new_message = new_message + sorted(temp, key = lambda val:val[0])[0][1] + " "
            else:
                new_message =  new_message + word + " "
        return new_message

    tqdm.pandas()
    print("Performing Spelling Corrections...")
    
    slangs = ["doge", "dogecoin", "dogecoins", "shib", "shiba", "shiba inu", "shibe inu", 
          "dollar", "dolar", "$", "ps", "p.s.", "app", "money", "tarde", "telegram", "whatsapp", 
          "buy", "issue", "crypto", "usdc", "bank", "account", "portfolio", "Elon", "Musk", "shibaa",
          "profit", "cro", "€", "inr", "mill", "cdc", "tbh", "hi", "hey", "plz", "wbu", "%",
          "crypto.com", "email", "usdt", "cent", "ct", "mil", "ppl", "btc", "curr"]

    a = [w for w in wordnet.all_lemma_names()]
    a = list(set(a).union(set(slangs)))
    correct_words = list(set(words.words()).union(set(a)))    
    dataframe['Messages'] = dataframe['Messages'].progress_apply(lambda txt: spell_check(txt, correct_words))
    return dataframe

def removing_non_english(dataframe): 
    def word_in_english(message, correct_words): 
        st = []
        for word in message.split(" "): 
            if wordnet.synsets(word) or word.lower() in correct_words:
                st.append(1)
            else: 
                st.append(0)

        cutoff_value = 0.45
        
        if sum(st)/len(st) >= cutoff_value: 
            return message
        else: 
            return ""
        
    tqdm.pandas()
    print("Checking English Messages...")
    slangs = ["doge", "dogecoin", "dogecoins", "shib", "shiba", "shiba inu", "shibe inu", 
              "dollar", "dolar", "$", "ps", "p.s.", "app", "money", "tarde", "telegram", "whatsapp", 
              "buy", "issue", "crypto", "usdc", "bank", "account", "portfolio", "Elon", "Musk", "shibaa",
              "profit", "cro", "€", "inr", "mill", "cdc", "tbh", "hi", "hey", "plz", "wbu", "%",
              "crypto.com", "email", "usdt", "cent", "ct", "mil", "ppl", "btc", "curr"]
    
    correct_words = list(set(words.words()).union(set(slangs)))    
    dataframe["Messages"] = dataframe["Messages"].progress_apply(lambda txt: word_in_english(txt, correct_words))
    return dataframe[dataframe['Messages'] > ""]

def remove_stop_words(dataframe): 

    def Remove_Stopwords(message, stop_words_list):
        tokens = message.split(" ")
        clean_message = [word for word in tokens if not word in stop_words_list]
        return [(" ").join(clean_message)]

    tqdm.pandas()
    print("Removing Stop Words...")
    deselect_stop_words = ['not', 'nor', 'no', 'against', 'don', "don't", 
          'should', "should've", 'aren', "aren't", 'couldn', 
          "couldn't", 'didn', "didn't", 'doesn', "doesn't", 
          'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 
          'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't", 
          'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 
          'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 
          'wouldn', "wouldn't"]

    stop_words_list = set([stopwords.words('english').remove(word) for word in deselect_stop_words])
    dataframe['Messages'] = dataframe['Messages'].progress_apply(lambda txt: Remove_Stopwords(txt, stop_words_list))
    return dataframe

def lemmatization(dataframe): 
    def lemmatize(message):
        word_lemma = WordNetLemmatizer()
        lemmatize_word = [word_lemma.lemmatize(word) for word in message]
        return (" ").join(lemmatize_word)
    
    tqdm.pandas()
    print("Lemmatizing Words in Messages...")
    dataframe['Messages']= dataframe['Messages'].progress_apply(lambda txt: lemmatize(txt))
    return dataframe

In [ ]:
df_pre = preprocessing(df)
df_pre

Decoding Emojis in Text...










  0%|                                                                         | 0/3061 [00:00<?, ?it/s]







  4%|██▌                                                          | 128/3061 [00:00<00:02, 1270.75it/s]







 11%|██████▍                                                      | 325/3061 [00:00<00:01, 1419.85it/s]







 18%|███████████▏                                                 | 562/3061 [00:00<00:01, 1611.50it/s]







 26%|███████████████▋                                             | 787/3061 [00:00<00:01, 1758.44it/s]







 32%|███████████████████▋                                         | 989/3061 [00:00<00:01, 1820.97it/s]







 39%|███████████████████████▍                                    | 1193/3061 [00:00<00:00, 1877.86it/s]







 45%|██████████████████████████▉                                 | 1377/3061 [00:00<00:00, 1862.19it/s]







 51%|██████████████████████████████▌                             | 1559/3061 [00:00<00:00, 1845.41it/s]


Converting Accented Characters...










  0%|                                                                         | 0/3061 [00:00<?, ?it/s]







100%|██████████████████████████████████████████████████████████| 3061/3061 [00:00<00:00, 609801.68it/s]

Removing Case Sensitive Characters...


...









  0%|                                                                         | 0/3061 [00:00<?, ?it/s]







100%|██████████████████████████████████████████████████████████| 3061/3061 [00:00<00:00, 235850.62it/s]

Lemmatizing Words in Messages...
Checking English Messages...










  0%|                                                                         | 0/3061 [00:00<?, ?it/s]







  0%|▏                                                                | 9/3061 [00:00<00:36, 83.52it/s]







  0%|▎                                                               | 15/3061 [00:00<00:41, 72.91it/s]







  1%|▍                                                               | 19/3061 [00:00<01:02, 48.90it/s]







  1%|▋                                                               | 30/3061 [00:00<00:54, 55.35it/s]







  1%|▊                                                               | 36/3061 [00:00<00:55, 54.69it/s]







  1%|▊                                                               | 41/3061 [00:00<01:02, 48.60it/s]







  2%|▉                                                               | 46/3061 [00:00<01:24, 35.89it/s]







  2%|█                                                               | 50/3061 [00:01<01:25, 35.10it/s]


Performing Spelling Corrections...










  0%|                                                                         | 0/2954 [00:00<?, ?it/s]







  0%|                                                                 | 2/2954 [00:01<29:19,  1.68it/s]







  0%|▏                                                                | 7/2954 [00:01<20:52,  2.35it/s]







  0%|▏                                                                | 9/2954 [00:02<21:16,  2.31it/s]







  0%|▏                                                               | 11/2954 [00:02<15:56,  3.08it/s]







  0%|▎                                                               | 13/2954 [00:07<50:19,  1.03s/it]







  0%|▎                                                               | 14/2954 [00:08<45:51,  1.07it/s]







  1%|▎                                                               | 15/2954 [00:09<43:58,  1.11it/s]







  1%|▎                                                             | 16/2954 [00:11<1:00:55,  1.24s/it]
